# How to pass variables between SoS steps

* **Difficulty level**: easy
* **Time need to lean**: 10 minutes or less
* **Key points**:

  

### Option `shared` <a id="Option_shared"></a>

SoS executes each step in a separate process and by default does not return any result to the master SoS process. Option `shared` is used to share variables between steps. This option accepts:

* A string (variable name), or
* A map between variable names and expressions (strings) that will be evaluated upon the completion of the step.
* A sequence of strings (variables) or maps.

For example,

In [1]:
%run
[10: shared='myvar']
myvar = 100

[20]
print(myvar)

100


A map syntax is recommended to share `step_output` of one step with others, because the variable assignment will be evaluated only after the step is complete:

In [2]:
%sandbox
%run
[1: shared = {'test_output': 'step_output'}]
output: 'a.txt'
sh:
    touch a.txt
[2]
print(f"Input file {test_output}")
input: test_output

Input file a.txt


The map syntax is evaluated as expressions; therefore it is possible to finer control what specific output, or variations of output, to share with others. For example:

In [3]:
%sandbox
%run
[1: shared={'test_output_1':'step_output[0]', 'test_output_2': 'step_output[1]'}]
output: 'a.txt', 'b.txt'
sh:
    touch a.txt b.txt
[2]
print(f"output 1: {test_output_1}")
print(f"output 2: {test_output_2}")

output 1: a.txt
output 2: b.txt


to shared the first file in `output` (filename `output[0]`) instead of the entire output file list.

The `shared` option also provides a `sos_variable` target.

Things becomes more complicated when there are multiple substeps. For example, when you use option `shared` on the following step with 10 substeps, only one of the random seed is returned because `rng` represent the last value of the variable after the completion of all substeps.

In [4]:
%run
[1: shared='seed']
input: for_each={'i': range(10)}
import random
seed = random.randint(0, 1000)


[2]
print(seed)

450


If you would like to see the variable in all substeps, you can prefix the variable name with `step_`

In [5]:
%run
[1: shared='step_seed']
input: for_each={'i': range(10)}
import random
seed = random.randint(0, 1000)


[2]
print(step_seed)

[858, 513, 328, 610, 142, 275, 458, 57, 762, 981]


You can also use the `step_*` vsriables in expressions as in the following example:

In [8]:
%run
[1: shared={'summed': 'sum(step_rng)', 'rngs': 'step_rng'}]
input: for_each={'i': range(10)}
import random
rng = random.randint(0, 10)


[2]
print(rngs)
print(summed)

[10, 0, 8, 1, 8, 9, 6, 7, 9, 1]
59


Variables generated by external tasks adds another layer of complexity because tasks usually do not share variables with the substep it belongs. To solve this problem, you will have to use the `shared` option of `task` to return the variable to the substep:

In [11]:
%run
[1: shared={'summed': 'sum(step_rng)', 'rngs': 'step_rng'}]
input: for_each={'i': range(10)}

task: shared='rng'
import random
rng = random.randint(0, 10*i)


[2]
print(rngs)
print(summed)

10 tasks completed: fc4e , b42d , ..., 559c


[0, 2, 12, 10, 24, 9, 51, 39, 79, 4]
230


## Further reading

* 